In [1]:
#Import all necessary libraries
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import imblearn
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import gc
import tqdm
import re
from collections import defaultdict
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency


import matplotlib.pyplot as plt
import plotly
import matplotlib.gridspec as gridspec
import sys

In [2]:
full_df = pd.read_csv('./Entire_df_merged.csv')

In [3]:
first = pd.read_csv('./First diagnosis.csv')

In [4]:
last = pd.read_csv('./Last Diagnosis.csv')

In [5]:
first.head()

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Expr5,Citizenship_Description,Gender_Name
0,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,160,Height,cm,48,EMIRATI,Male
1,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,24,BMI (Body Mass Index),kg/m2,48,EMIRATI,Male
2,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,36.5,Body Temperature,DegC,48,EMIRATI,Male
3,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,75,Heart Rate,bpm,48,EMIRATI,Male
4,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,24,Respiratory Rate,bpm,48,EMIRATI,Male


In [6]:
first.shape, last.shape

((100000, 12), (100000, 12))

In [7]:
first.rename(columns = {'Expr5': 'Age', 'Gender_Name': 'Gender', 'Citizenship_Description': 'Citizen'}, inplace = True)
last.rename(columns = {'Expr5': 'Age', 'Gender_Name': 'Gender', 'Citizenship_Description': 'Citizen'}, inplace = True)


In [8]:
def split_col(data):
    data['Height (in cm)'] = data.apply(lambda x: x['ObservationValue'] if 'Height' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Weight (in kg)'] = data.apply(lambda x: x['ObservationValue'] if 'Weight' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['BMI in kg/m2'] = data.apply(lambda x: x['ObservationValue'] if 'BMI (Body Mass Index)' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Body Temperature (in DegC)'] = data.apply(lambda x: x['ObservationValue'] if 'Body Temperature' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Diastolic Blood Pressure in mmHg'] = data.apply(lambda x: x['ObservationValue'] if 'Diastolic Blood Pressure' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Systolic Blood Pressure in mmHg'] = data.apply(lambda x: x['ObservationValue'] if 'Systolic Blood Pressure' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Heart Rate in bpm'] = data.apply(lambda x: x['ObservationValue'] if 'Heart Rate' in x['ObservationCode_Description'] else np.nan, axis=1)
    data['Respiratory Rate in bpm'] = data.apply(lambda x: x['ObservationValue'] if 'Respiratory Rate' in x['ObservationCode_Description'] else np.nan, axis=1)   

    return data

In [9]:
split_col(first)

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,160,Height,cm,48,EMIRATI,Male,160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,24,BMI (Body Mass Index),kg/m2,48,EMIRATI,Male,NaN,NaN,24,NaN,NaN,NaN,NaN,NaN
2,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,36.5,Body Temperature,DegC,48,EMIRATI,Male,NaN,NaN,NaN,36.5,NaN,NaN,NaN,NaN
3,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,75,Heart Rate,bpm,48,EMIRATI,Male,NaN,NaN,NaN,NaN,NaN,NaN,75,NaN
4,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,24,Respiratory Rate,bpm,48,EMIRATI,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1127259,U07.1,Normal,ABSENT,NaN,OTHERS,88,Heart Rate,bpm,42,UAE,Male,NaN,NaN,NaN,NaN,NaN,NaN,88,NaN
99996,1127259,U07.1,Normal,5,NaN,pH,88,Heart Rate,bpm,42,UAE,Male,NaN,NaN,NaN,NaN,NaN,NaN,88,NaN
99997,1127259,U07.1,Normal,TRACE,NaN,PROTEIN,88,Heart Rate,bpm,42,UAE,Male,NaN,NaN,NaN,NaN,NaN,NaN,88,NaN
99998,1127259,U07.1,Normal,2-4/HPF,NaN,RED BLOOD CELLS,88,Heart Rate,bpm,42,UAE,Male,NaN,NaN,NaN,NaN,NaN,NaN,88,NaN


In [10]:
def split_col2(data):
    data['Height (in cm)'] = data.apply(lambda x: x['ObservationValue'] if 'Height' in str(x['ObservationCode_Description']) else np.nan, axis=1)
    data['Weight (in kg)'] = data.apply(lambda x: x['ObservationValue'] if 'Weight' in str(x['ObservationCode_Description']) else np.nan, axis=1)
    data['BMI in kg/m2'] = data.apply(lambda x: x['ObservationValue'] if 'BMI' in str(x['ObservationCode_Description']) else np.nan, axis=1)
    data['Body Temperature (in DegC)'] = data.apply(lambda x: x['ObservationValue'] if 'Body Temperature' in str(x['ObservationCode_Description']) else np.nan, axis=1)
    data['Diastolic Blood Pressure in mmHg'] = data.apply(lambda x: x['ObservationValue'] if 'Diastolic Blood Pressure' in str(x['ObservationCode_Description']) else np.nan, axis=1)
    data['Systolic Blood Pressure in mmHg'] = data.apply(lambda x: x['ObservationValue'] if 'Systolic Blood Pressure' in str(x['ObservationCode_Description']) else np.nan, axis=1)
    data['Heart Rate in bpm'] = data.apply(lambda x: x['ObservationValue'] if 'Heart Rate' in str(x['ObservationCode_Description']) else np.nan, axis=1)
    data['Respiratory Rate in bpm'] = data.apply(lambda x: x['ObservationValue'] if 'Respiratory Rate' in str(x['ObservationCode_Description']) else np.nan, axis=1)
  
    return data


In [11]:
split_col2(last)

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,131620,Z98.890,NaN,0.4,NaN,Hgb F %,24,BMI (Body Mass Index),kg/m2,48,EMIRATI,Male,NaN,NaN,24,NaN,NaN,NaN,NaN,NaN
1,131620,Z98.890,NaN,85.9,NaN,Hgb A %,24,BMI (Body Mass Index),kg/m2,48,EMIRATI,Male,NaN,NaN,24,NaN,NaN,NaN,NaN,NaN
2,131620,Z98.890,NaN,2.5,NaN,Hgb A2 %,24,BMI (Body Mass Index),kg/m2,48,EMIRATI,Male,NaN,NaN,24,NaN,NaN,NaN,NaN,NaN
3,131620,Z98.890,Normal,Normal Haemaglobin Patter,NaN,Hgb Interp,24,BMI (Body Mass Index),kg/m2,48,EMIRATI,Male,NaN,NaN,24,NaN,NaN,NaN,NaN,NaN
4,131620,E11.42,NaN,0.4,NaN,Hgb F %,24,BMI (Body Mass Index),kg/m2,48,EMIRATI,Male,NaN,NaN,24,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,578968,J06.9,Normal,5.11,x 10\S\12/L,RBC count,80.0,Diastolic Blood Pressure,mmHg,29,Indian,Male,NaN,NaN,NaN,NaN,80.0,NaN,NaN,NaN
99996,578968,J06.9,Normal,267,x 10\S\9/L,Platelet count,80.0,Diastolic Blood Pressure,mmHg,29,Indian,Male,NaN,NaN,NaN,NaN,80.0,NaN,NaN,NaN
99997,578968,E11.9,Normal,44,%,Haematocrit,80.0,Diastolic Blood Pressure,mmHg,29,Indian,Male,NaN,NaN,NaN,NaN,80.0,NaN,NaN,NaN
99998,578968,E11.9,Normal,9.4,%,Monocyte,80.0,Diastolic Blood Pressure,mmHg,29,Indian,Male,NaN,NaN,NaN,NaN,80.0,NaN,NaN,NaN


In [12]:
first_1 = first.groupby('Patient').apply(lambda x: x.sample(1)).reset_index(drop=True)
last_1 = last.groupby('Patient').apply(lambda x: x.sample(1)).reset_index(drop=True)

In [13]:
first_1.shape

(909, 20)

In [14]:
obs_col = ['Height (in cm)', 'Weight (in kg)', 'BMI in kg/m2', 'Body Temperature (in DegC)', 'Diastolic Blood Pressure in mmHg', 'Systolic Blood Pressure in mmHg', 'Heart Rate in bpm', 'Respiratory Rate in bpm']

In [15]:
def conv_to_num(obs_col, data):
    for i in obs_col:
        data[i] = pd.to_numeric(data[i], errors = "coerce")
    return data

In [16]:
conv_to_num(obs_col, first_1)

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,120,Systolic Blood Pressure,mmHg,48,EMIRATI,Male,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN
1,132534,R52,NaN,9.7,fL,PDW,146,Systolic Blood Pressure,mmHg,68,EMIRATI,Female,NaN,NaN,NaN,NaN,NaN,146.0,NaN,NaN
2,132691,J02.9,Normal,Not Detected,NaN,COVID-19 PCR,29,BMI (Body Mass Index),kg/m2,44,EMIRATI,Female,NaN,NaN,29.0,NaN,NaN,NaN,NaN,NaN
3,132738,Z32.00,Normal,Not Detected,NaN,COVID-19 PCR,36.4,Body Temperature,DegC,43,EMIRATI,Female,NaN,NaN,NaN,36.4,NaN,NaN,NaN,NaN
4,133122,L85.3,Low,11,gm/dL,Hgb,75,Heart Rate,bpm,41,EMIRATI,Female,NaN,NaN,NaN,NaN,NaN,NaN,75.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,1826401,R10.84,Normal,49,/cumm,BASOPHILS Absolute count,90,Diastolic Blood Pressure,mmHg,46,PAKISTANI,Male,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN
905,1831406,E78.5,Normal,3.2,pg/mL,TROPONIN T HIGH SENSITIVE,169,Height,cm,55,Indian,Male,169.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
906,1832052,R42,Normal,6.27,NaN,WBC,18,Respiratory Rate,bpm,31,Indian,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0
907,1832617,M79.672,Normal,0.88,mg/dl,"Serum ,Creatinine",98,SpO2,%,42,Indian,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
conv_to_num(obs_col, last_1)

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,131620,Z98.890,NaN,2.5,NaN,Hgb A2 %,36.7,Body Temperature,DegC,48,EMIRATI,Male,NaN,NaN,NaN,36.7,NaN,NaN,NaN,NaN
1,132534,M17.10,Normal,3.9,mmol/L,"POTASSIUM , SERUM",145,Systolic Blood Pressure,mmHg,68,EMIRATI,Female,NaN,NaN,NaN,NaN,NaN,145.0,NaN,NaN
2,132691,J34.2,Normal,14.85,pmol/L,T4 Free,81.0,Weight,kg,44,EMIRATI,Female,NaN,81.0,NaN,NaN,NaN,NaN,NaN,NaN
3,132738,E66.9,Normal,107.7,ZZ,eGFR,36.6,Body Temperature,DegC,43,EMIRATI,Female,NaN,NaN,NaN,36.6,NaN,NaN,NaN,NaN
4,133122,E03.8,Normal,4.3,mg/L,Ur Microalbumin,99,Weight,kg,41,EMIRATI,Female,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509,3255368,R09.3,High,152,pg/mL,Natriuretic peptide (pro-BNP)*,150.0,Systolic Blood Pressure,mmHg,70,Comoros,Female,NaN,NaN,NaN,NaN,NaN,150.0,NaN,NaN
1510,3256965,O99.810,Normal,5.8,%,HBA1C-GLYCATED HAEMOGLOBIN,100.0,SpO2,%,37,Uae,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1511,3259793,L55.0,Normal,12.9,%,* RDWcv,20.0,Respiratory Rate,bpm,47,Jordanian,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
1512,3268774,R21,Normal,5.5,Percent,HbA1c,41.19,BMI (Body Mass Index),kg/m2,52,Pakistani,Female,NaN,NaN,41.19,NaN,NaN,NaN,NaN,NaN


In [18]:
def extract_values(data, first):
    first_1 = data.copy()

    for i in first_1['Patient'].unique():
        patient_data = first[first['Patient'] == i]
        
        for _, row in patient_data.iterrows():
            desc = row['ObservationCode_Description']
            value = row['ObservationValue']
            
            if 'Height' in desc:
                first_1.loc[first_1['Patient'] == i, 'Height (in cm)'] = value
            elif 'Weight' in desc:
                first_1.loc[first_1['Patient'] == i, 'Weight (in kg)'] = value
            elif 'BMI' in desc:
                first_1.loc[first_1['Patient'] == i, 'BMI in kg/m2'] = value
            elif 'Body Temperature' in desc:
                first_1.loc[first_1['Patient'] == i, 'Body Temperature (in DegC)'] = value
            elif 'Diastolic Blood Pressure' in desc:
                first_1.loc[first_1['Patient'] == i, 'Diastolic Blood Pressure in mmHg'] = value
            elif 'Systolic Blood Pressure' in desc:
                first_1.loc[first_1['Patient'] == i, 'Systolic Blood Pressure in mmHg'] = value
            elif 'Heart Rate' in desc:
                first_1.loc[first_1['Patient'] == i, 'Heart Rate in bpm'] = value
            elif 'Respiratory Rate' in desc:
                first_1.loc[first_1['Patient'] == i, 'Respiratory Rate in bpm'] = value

    return first_1

In [19]:
first_1 = extract_values(first_1, first)

In [20]:
first_1.head()

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,120,Systolic Blood Pressure,mmHg,48,EMIRATI,Male,160,62,24,36.5,84,120,75,24
1,132534,R52,NaN,9.7,fL,PDW,146,Systolic Blood Pressure,mmHg,68,EMIRATI,Female,NaN,NaN,NaN,36.4,84,146,74,18
2,132691,J02.9,Normal,Not Detected,NaN,COVID-19 PCR,29,BMI (Body Mass Index),kg/m2,44,EMIRATI,Female,162,76,29,36.3,80,120,88,16
3,132738,Z32.00,Normal,Not Detected,NaN,COVID-19 PCR,36.4,Body Temperature,DegC,43,EMIRATI,Female,NaN,NaN,NaN,36.4,70,110,72,NaN
4,133122,L85.3,Low,11,gm/dL,Hgb,75,Heart Rate,bpm,41,EMIRATI,Female,NaN,NaN,NaN,36.8,NaN,NaN,75,NaN


In [21]:
def extract_values2(data, last):
    last_1 = data.copy()

    for i in last_1['Patient'].unique():
        patient_data = last[last['Patient'] == i]
        
        for _, row in patient_data.iterrows():
            desc = str(row['ObservationCode_Description'])
            value = row['ObservationValue']
            
            if 'Height' in desc:
                last_1.loc[last_1['Patient'] == i, 'Height (in cm)'] = value
            elif 'Weight' in desc:
                last_1.loc[last_1['Patient'] == i, 'Weight (in kg)'] = value
            elif 'BMI' in desc:
                last_1.loc[last_1['Patient'] == i, 'BMI in kg/m2'] = value
            elif 'Body Temperature' in desc:
                last_1.loc[last_1['Patient'] == i, 'Body Temperature (in DegC)'] = value
            elif 'Diastolic Blood Pressure' in desc:
                last_1.loc[last_1['Patient'] == i, 'Diastolic Blood Pressure in mmHg'] = value
            elif 'Systolic Blood Pressure' in desc:
                last_1.loc[last_1['Patient'] == i, 'Systolic Blood Pressure in mmHg'] = value
            elif 'Heart Rate' in desc:
                last_1.loc[last_1['Patient'] == i, 'Heart Rate in bpm'] = value
            elif 'Respiratory Rate' in desc:
                last_1.loc[last_1['Patient'] == i, 'Respiratory Rate in bpm'] = value

    return last_1


In [22]:
last_1 = extract_values2(last_1, last)
last_1.head()

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationValue,ObservationCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,131620,Z98.890,NaN,2.5,NaN,Hgb A2 %,36.7,Body Temperature,DegC,48,EMIRATI,Male,161,61.2,24,36.7,68,108,72,20
1,132534,M17.10,Normal,3.9,mmol/L,"POTASSIUM , SERUM",145,Systolic Blood Pressure,mmHg,68,EMIRATI,Female,145,47,22,36.6,85,145,88,20
2,132691,J34.2,Normal,14.85,pmol/L,T4 Free,81.0,Weight,kg,44,EMIRATI,Female,161.0,81.0,31.0,36.7,85.0,131.0,61.0,18.0
3,132738,E66.9,Normal,107.7,ZZ,eGFR,36.6,Body Temperature,DegC,43,EMIRATI,Female,154,79.6,34,36.6,83,109,78,18
4,133122,E03.8,Normal,4.3,mg/L,Ur Microalbumin,99,Weight,kg,41,EMIRATI,Female,154,99,42,36.8,79,110,83,18


In [23]:
#we will now merge the first and last dataframes
first_1 = first_1.drop(['ObservationCode_Description', 'ObservationValue'], axis=1)
last_1 = last_1.drop(['ObservationCode_Description', 'ObservationValue'], axis=1)

In [24]:
first_1.shape, last_1.shape

((909, 18), (1514, 18))

In [25]:
obs_col = ['Height (in cm)', 'Weight (in kg)', 'BMI in kg/m2', 'Body Temperature (in DegC)', 'Diastolic Blood Pressure in mmHg', 'Systolic Blood Pressure in mmHg', 'Heart Rate in bpm', 'Respiratory Rate in bpm']

In [26]:
def conv_to_num(obs_col, data):
    for i in obs_col:
        data[i] = pd.to_numeric(data[i], errors = "coerce")
    return data

In [27]:
conv_to_num(obs_col, first_1)

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,mmHg,48,EMIRATI,Male,160.0,62.0,24.0,36.5,84.0,120.0,75.0,24.0
1,132534,R52,NaN,9.7,fL,PDW,mmHg,68,EMIRATI,Female,NaN,NaN,NaN,36.4,84.0,146.0,74.0,18.0
2,132691,J02.9,Normal,Not Detected,NaN,COVID-19 PCR,kg/m2,44,EMIRATI,Female,162.0,76.0,29.0,36.3,80.0,120.0,88.0,16.0
3,132738,Z32.00,Normal,Not Detected,NaN,COVID-19 PCR,DegC,43,EMIRATI,Female,NaN,NaN,NaN,36.4,70.0,110.0,72.0,NaN
4,133122,L85.3,Low,11,gm/dL,Hgb,bpm,41,EMIRATI,Female,NaN,NaN,NaN,36.8,NaN,NaN,75.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,1826401,R10.84,Normal,49,/cumm,BASOPHILS Absolute count,mmHg,46,PAKISTANI,Male,NaN,83.0,NaN,36.8,90.0,148.0,106.0,20.0
905,1831406,E78.5,Normal,3.2,pg/mL,TROPONIN T HIGH SENSITIVE,cm,55,Indian,Male,169.0,64.0,NaN,NaN,78.0,122.0,72.0,NaN
906,1832052,R42,Normal,6.27,NaN,WBC,bpm,31,Indian,Male,175.0,71.0,NaN,NaN,73.0,127.0,71.0,18.0
907,1832617,M79.672,Normal,0.88,mg/dl,"Serum ,Creatinine",%,42,Indian,Male,NaN,103.8,NaN,36.1,65.0,110.0,78.0,21.0


In [28]:
conv_to_num(obs_col, last_1)

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
0,131620,Z98.890,NaN,2.5,NaN,Hgb A2 %,DegC,48,EMIRATI,Male,161.0,61.20,24.00,36.7,68.0,108.00,72.0,20.0
1,132534,M17.10,Normal,3.9,mmol/L,"POTASSIUM , SERUM",mmHg,68,EMIRATI,Female,145.0,47.00,22.00,36.6,85.0,145.00,88.0,20.0
2,132691,J34.2,Normal,14.85,pmol/L,T4 Free,kg,44,EMIRATI,Female,161.0,81.00,31.00,36.7,85.0,131.00,61.0,18.0
3,132738,E66.9,Normal,107.7,ZZ,eGFR,DegC,43,EMIRATI,Female,154.0,79.60,34.00,36.6,83.0,109.00,78.0,18.0
4,133122,E03.8,Normal,4.3,mg/L,Ur Microalbumin,kg,41,EMIRATI,Female,154.0,99.00,42.00,36.8,79.0,110.00,83.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509,3255368,R09.3,High,152,pg/mL,Natriuretic peptide (pro-BNP)*,mmHg,70,Comoros,Female,NaN,NaN,NaN,36.2,95.0,150.00,90.0,NaN
1510,3256965,O99.810,Normal,5.8,%,HBA1C-GLYCATED HAEMOGLOBIN,%,37,Uae,Female,154.0,91.70,39.00,36.6,78.0,128.00,NaN,20.0
1511,3259793,L55.0,Normal,12.9,%,* RDWcv,bpm,47,Jordanian,Female,NaN,80.00,NaN,36.5,60.0,100.00,68.0,20.0
1512,3268774,R21,Normal,5.5,Percent,HbA1c,kg/m2,52,Pakistani,Female,167.5,115.55,41.19,36.6,69.0,121.00,81.0,15.0


In [29]:
def null_columns(data):
    null_df = pd.DataFrame(data.isnull().sum(), columns = ["Null Values"])
    null_df["% Null Values"] = round((null_df["Null Values"]/len(data))*100, 2)
    return null_df[null_df["Null Values"] > 0]

In [30]:
null_columns(last_1).sort_values(by = "Null Values", ascending = False)

,Null Values,% Null Values
BMI in kg/m2,388,25.63
Respiratory Rate in bpm,363,23.98
ResultInterpretation,362,23.91
Height (in cm),288,19.02
Body Temperature (in DegC),245,16.18
Heart Rate in bpm,231,15.26
ResultValueUnits,224,14.80
Weight (in kg),107,7.07
Diastolic Blood Pressure in mmHg,99,6.54
Systolic Blood Pressure in mmHg,87,5.75


In [31]:
null_columns(first_1).sort_values(by = "Null Values", ascending = False)

,Null Values,% Null Values
BMI in kg/m2,443,48.73
ResultValueUnits,418,45.98
Height (in cm),299,32.89
Respiratory Rate in bpm,225,24.75
Weight (in kg),135,14.85
Body Temperature (in DegC),102,11.22
ResultInterpretation,72,7.92
Heart Rate in bpm,48,5.28
Systolic Blood Pressure in mmHg,39,4.29
Diastolic Blood Pressure in mmHg,37,4.07


In [32]:
#Calculating the BMI from height and weight
first_1['BMI in kg/m2'] = first_1['BMI in kg/m2'].fillna(first_1['Weight (in kg)']/(first_1['Height (in cm)']/100)**2)
last_1['BMI in kg/m2'] = last_1['BMI in kg/m2'].fillna(last_1['Weight (in kg)']/(last_1['Height (in cm)']/100)**2)

In [33]:
#Now we will create a new column with unique ID for each patient, so that we can map the patient to the actual target after prediction

#since the variable Patient is int type, we will convert it to string
first_1['Patient'] = first_1['Patient'].astype(str)
last_1['Patient'] = last_1['Patient'].astype(str)

In [34]:
#now we will generate a unique ID for each patient so that we can map the patient to the actual target after prediction

first_1['ID'] = first_1['Patient'].apply(lambda x: 'P_' + x)

In [35]:
first_1.head()

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm,ID
0,131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,mmHg,48,EMIRATI,Male,160.0,62.0,24.0,36.5,84.0,120.0,75.0,24.0,P_131620
1,132534,R52,NaN,9.7,fL,PDW,mmHg,68,EMIRATI,Female,NaN,NaN,NaN,36.4,84.0,146.0,74.0,18.0,P_132534
2,132691,J02.9,Normal,Not Detected,NaN,COVID-19 PCR,kg/m2,44,EMIRATI,Female,162.0,76.0,29.0,36.3,80.0,120.0,88.0,16.0,P_132691
3,132738,Z32.00,Normal,Not Detected,NaN,COVID-19 PCR,DegC,43,EMIRATI,Female,NaN,NaN,NaN,36.4,70.0,110.0,72.0,NaN,P_132738
4,133122,L85.3,Low,11,gm/dL,Hgb,bpm,41,EMIRATI,Female,NaN,NaN,NaN,36.8,NaN,NaN,75.0,NaN,P_133122


In [36]:
#How will we map the patient to the actual target after prediction?
#We will use the ID column to map the patient to the actual target after prediction
#we will use the ID column as the index of the dataframe 
first_1.set_index('ID', inplace = True)


In [37]:
last_1['ID'] = last_1['Patient'].apply(lambda x: 'P_' + x)
last_1.set_index('ID', inplace = True)
last_1.head()

,Patient,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,ObservationCode_ObservationValueUnits_Code,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
ID,,,,,,,,,,,,,,,,,,
P_131620,131620,Z98.890,NaN,2.5,NaN,Hgb A2 %,DegC,48,EMIRATI,Male,161.0,61.2,24.0,36.7,68.0,108.0,72.0,20.0
P_132534,132534,M17.10,Normal,3.9,mmol/L,"POTASSIUM , SERUM",mmHg,68,EMIRATI,Female,145.0,47.0,22.0,36.6,85.0,145.0,88.0,20.0
P_132691,132691,J34.2,Normal,14.85,pmol/L,T4 Free,kg,44,EMIRATI,Female,161.0,81.0,31.0,36.7,85.0,131.0,61.0,18.0
P_132738,132738,E66.9,Normal,107.7,ZZ,eGFR,DegC,43,EMIRATI,Female,154.0,79.6,34.0,36.6,83.0,109.0,78.0,18.0
P_133122,133122,E03.8,Normal,4.3,mg/L,Ur Microalbumin,kg,41,EMIRATI,Female,154.0,99.0,42.0,36.8,79.0,110.0,83.0,18.0


In [38]:
#we will drop the Patient column from both the dataframes and the observation code column from the last_1 dataframe
first_1.drop(['Patient'], axis = 1, inplace = True)
last_1.drop(['Patient', 'ObservationCode_ObservationValueUnits_Code'], axis = 1, inplace = True)

In [39]:
first_1.drop(['ObservationCode_ObservationValueUnits_Code'], axis = 1, inplace = True)

In [40]:
first_1.head()  

,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm
ID,,,,,,,,,,,,,,,,
P_131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,48,EMIRATI,Male,160.0,62.0,24.0,36.5,84.0,120.0,75.0,24.0
P_132534,R52,NaN,9.7,fL,PDW,68,EMIRATI,Female,NaN,NaN,NaN,36.4,84.0,146.0,74.0,18.0
P_132691,J02.9,Normal,Not Detected,NaN,COVID-19 PCR,44,EMIRATI,Female,162.0,76.0,29.0,36.3,80.0,120.0,88.0,16.0
P_132738,Z32.00,Normal,Not Detected,NaN,COVID-19 PCR,43,EMIRATI,Female,NaN,NaN,NaN,36.4,70.0,110.0,72.0,NaN
P_133122,L85.3,Low,11,gm/dL,Hgb,41,EMIRATI,Female,NaN,NaN,NaN,36.8,NaN,NaN,75.0,NaN


In [41]:
first_1["ResultInterpretation"].unique()

array(['Normal', nan, 'Low', 'High', 'Critical Low', 'I', 'Critical High',
       'Abnormal'], dtype=object)

In [42]:
first_1['Normal'] = np.where(first_1['ResultInterpretation'] == 'Normal', 1, 0)
first_1['Low'] = np.where(first_1['ResultInterpretation'] == 'Low', 1, 0)
first_1['High'] = np.where(first_1['ResultInterpretation'] == 'High', 1, 0)
first_1['Critical Low'] = np.where(first_1['ResultInterpretation'] == 'Critical Low', 1, 0)
first_1['Abnormal'] = np.where(first_1['ResultInterpretation'] == 'Abnormal', 1, 0)
first_1['Critical High'] = np.where(first_1['ResultInterpretation'] == 'Critical High', 1, 0)
first_1['I'] = np.where(first_1['ResultInterpretation'] == 'I', 1, 0)



In [43]:
first_1.head()

,Diagnosis_Code,ResultInterpretation,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Citizen,Gender,Height (in cm),Weight (in kg),...,Systolic Blood Pressure in mmHg,Heart Rate in bpm,Respiratory Rate in bpm,Normal,Low,High,Critical Low,Abnormal,Critical High,I
ID,,,,,,,,,,,,,,,,,,,,,
P_131620,Z23,Normal,Not Detected,NaN,COVID-19 PCR,48,EMIRATI,Male,160.0,62.0,...,120.0,75.0,24.0,1,0,0,0,0,0,0
P_132534,R52,NaN,9.7,fL,PDW,68,EMIRATI,Female,NaN,NaN,...,146.0,74.0,18.0,0,0,0,0,0,0,0
P_132691,J02.9,Normal,Not Detected,NaN,COVID-19 PCR,44,EMIRATI,Female,162.0,76.0,...,120.0,88.0,16.0,1,0,0,0,0,0,0
P_132738,Z32.00,Normal,Not Detected,NaN,COVID-19 PCR,43,EMIRATI,Female,NaN,NaN,...,110.0,72.0,NaN,1,0,0,0,0,0,0
P_133122,L85.3,Low,11,gm/dL,Hgb,41,EMIRATI,Female,NaN,NaN,...,NaN,75.0,NaN,0,1,0,0,0,0,0


In [44]:
first_1.drop(['ResultInterpretation'], axis = 1, inplace = True)

In [45]:
#We will split the DiagnosisCode column into 3 columns, category, subcategory and extension 
#where Z02.89: Z is the category, 02 is the subcategory and 89 is the extension

first_1['Category'] = first_1['Diagnosis_Code'].str[0]
first_1['Subcategory'] = first_1['Diagnosis_Code'].str[1:3]
first_1['Extension'] = first_1['Diagnosis_Code'].str[3:]

first_1.head()




,Diagnosis_Code,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,...,Normal,Low,High,Critical Low,Abnormal,Critical High,I,Category,Subcategory,Extension
ID,,,,,,,,,,,,,,,,,,,,,
P_131620,Z23,Not Detected,NaN,COVID-19 PCR,48,EMIRATI,Male,160.0,62.0,24.0,...,1,0,0,0,0,0,0,Z,23,
P_132534,R52,9.7,fL,PDW,68,EMIRATI,Female,NaN,NaN,NaN,...,0,0,0,0,0,0,0,R,52,
P_132691,J02.9,Not Detected,NaN,COVID-19 PCR,44,EMIRATI,Female,162.0,76.0,29.0,...,1,0,0,0,0,0,0,J,02,.9
P_132738,Z32.00,Not Detected,NaN,COVID-19 PCR,43,EMIRATI,Female,NaN,NaN,NaN,...,1,0,0,0,0,0,0,Z,32,.00
P_133122,L85.3,11,gm/dL,Hgb,41,EMIRATI,Female,NaN,NaN,NaN,...,0,1,0,0,0,0,0,L,85,.3


In [46]:
first_1.drop(['Diagnosis_Code'], axis = 1, inplace = True)

In [47]:
first_1.head()

,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Citizen,Gender,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),...,Normal,Low,High,Critical Low,Abnormal,Critical High,I,Category,Subcategory,Extension
ID,,,,,,,,,,,,,,,,,,,,,
P_131620,Not Detected,NaN,COVID-19 PCR,48,EMIRATI,Male,160.0,62.0,24.0,36.5,...,1,0,0,0,0,0,0,Z,23,
P_132534,9.7,fL,PDW,68,EMIRATI,Female,NaN,NaN,NaN,36.4,...,0,0,0,0,0,0,0,R,52,
P_132691,Not Detected,NaN,COVID-19 PCR,44,EMIRATI,Female,162.0,76.0,29.0,36.3,...,1,0,0,0,0,0,0,J,02,.9
P_132738,Not Detected,NaN,COVID-19 PCR,43,EMIRATI,Female,NaN,NaN,NaN,36.4,...,1,0,0,0,0,0,0,Z,32,.00
P_133122,11,gm/dL,Hgb,41,EMIRATI,Female,NaN,NaN,NaN,36.8,...,0,1,0,0,0,0,0,L,85,.3


In [48]:
#We will one hot encode the gender column

first_1 = pd.get_dummies(columns=['Gender'], prefix='Gender', prefix_sep='_', data=first_1, dummy_na=True, drop_first=False, dtype=None)

In [49]:
first_1.head()

,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Citizen,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,...,Critical Low,Abnormal,Critical High,I,Category,Subcategory,Extension,Gender_Female,Gender_Male,Gender_nan
ID,,,,,,,,,,,,,,,,,,,,,
P_131620,Not Detected,NaN,COVID-19 PCR,48,EMIRATI,160.0,62.0,24.0,36.5,84.0,...,0,0,0,0,Z,23,,False,True,False
P_132534,9.7,fL,PDW,68,EMIRATI,NaN,NaN,NaN,36.4,84.0,...,0,0,0,0,R,52,,True,False,False
P_132691,Not Detected,NaN,COVID-19 PCR,44,EMIRATI,162.0,76.0,29.0,36.3,80.0,...,0,0,0,0,J,02,.9,True,False,False
P_132738,Not Detected,NaN,COVID-19 PCR,43,EMIRATI,NaN,NaN,NaN,36.4,70.0,...,0,0,0,0,Z,32,.00,True,False,False
P_133122,11,gm/dL,Hgb,41,EMIRATI,NaN,NaN,NaN,36.8,NaN,...,0,0,0,0,L,85,.3,True,False,False


In [50]:
#rename the columns
first_1.rename(columns={'Gender_nan':'Gender_Unknown'})

,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Citizen,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,...,Critical Low,Abnormal,Critical High,I,Category,Subcategory,Extension,Gender_Female,Gender_Male,Gender_Unknown
ID,,,,,,,,,,,,,,,,,,,,,
P_131620,Not Detected,NaN,COVID-19 PCR,48,EMIRATI,160.0,62.0,24.000000,36.5,84.0,...,0,0,0,0,Z,23,,False,True,False
P_132534,9.7,fL,PDW,68,EMIRATI,NaN,NaN,NaN,36.4,84.0,...,0,0,0,0,R,52,,True,False,False
P_132691,Not Detected,NaN,COVID-19 PCR,44,EMIRATI,162.0,76.0,29.000000,36.3,80.0,...,0,0,0,0,J,02,.9,True,False,False
P_132738,Not Detected,NaN,COVID-19 PCR,43,EMIRATI,NaN,NaN,NaN,36.4,70.0,...,0,0,0,0,Z,32,.00,True,False,False
P_133122,11,gm/dL,Hgb,41,EMIRATI,NaN,NaN,NaN,36.8,NaN,...,0,0,0,0,L,85,.3,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P_1826401,49,/cumm,BASOPHILS Absolute count,46,PAKISTANI,NaN,83.0,NaN,36.8,90.0,...,0,0,0,0,R,10,.84,False,True,False
P_1831406,3.2,pg/mL,TROPONIN T HIGH SENSITIVE,55,Indian,169.0,64.0,22.408179,NaN,78.0,...,0,0,0,0,E,78,.5,False,True,False
P_1832052,6.27,NaN,WBC,31,Indian,175.0,71.0,23.183673,NaN,73.0,...,0,0,0,0,R,42,,False,True,False


In [64]:
#We will now map the Citizen column to the actual country name

citizen_names = pd.read_csv('./citizen.csv')

In [65]:
citizen_names.head()

,Citizen,Country
0,Citizen_AFGHANISTAN,AFGHANISTAN
1,Citizen_ALGERIA,ALGERIA
2,Citizen_AMERICAN,AMERICAN
3,Citizen_American,AMERICAN
4,Citizen_Andorran,ANDORRAN


In [66]:
#We will one hot encode the Citizen column

first_1 = pd.get_dummies(columns=['Citizen'], prefix='Citizen', prefix_sep='_', data=first_1, dummy_na=True, drop_first=False, dtype=None)

KeyError: "None of [Index(['Citizen'], dtype='object')] are in the [columns]"

In [67]:
first_1.head()

,ResultValue,ResultValueUnits,TestItemCode_Description,Age,Height (in cm),Weight (in kg),BMI in kg/m2,Body Temperature (in DegC),Diastolic Blood Pressure in mmHg,Systolic Blood Pressure in mmHg,...,Citizen_UAE,Citizen_UNITED ARAB EMIRATES,Citizen_UNITED STATE OF AMERICA,Citizen_Uae,Citizen_Unknown,Citizen_YEMEN,Citizen_YEMENI,Citizen_Yemeni,Citizen_Zimbabwean,Citizen_nan
ID,,,,,,,,,,,,,,,,,,,,,
P_131620,Not Detected,NaN,COVID-19 PCR,48,160.0,62.0,24.0,36.5,84.0,120.0,...,False,False,False,False,False,False,False,False,False,False
P_132534,9.7,fL,PDW,68,NaN,NaN,NaN,36.4,84.0,146.0,...,False,False,False,False,False,False,False,False,False,False
P_132691,Not Detected,NaN,COVID-19 PCR,44,162.0,76.0,29.0,36.3,80.0,120.0,...,False,False,False,False,False,False,False,False,False,False
P_132738,Not Detected,NaN,COVID-19 PCR,43,NaN,NaN,NaN,36.4,70.0,110.0,...,False,False,False,False,False,False,False,False,False,False
P_133122,11,gm/dL,Hgb,41,NaN,NaN,NaN,36.8,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
